In [1]:
# Import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# Create hopteaming database.
db = sqlite3.connect('data/hopteaming.sqlite')
cursor = db.cursor()
cursor.executescript("""
DROP TABLE IF EXISTS hopteaming;
DROP TABLE IF EXISTS nppes;
""")

In [3]:
# Create list of columns to use for nppes dataset.
list_of_nppes_cols = ['NPI',
                      'Entity Type Code',
                      'Provider Organization Name (Legal Business Name)',
                      'Provider Last Name (Legal Name)',
                      'Provider First Name',
                      'Provider Middle Name',
                      'Provider Name Prefix Text',
                      'Provider Name Suffix Text',
                      'Provider Credential Text',
                      'Provider First Line Business Practice Location Address',
                      'Provider Second Line Business Practice Location Address',
                      'Provider Business Practice Location Address City Name',
                      'Provider Business Practice Location Address State Name',
                      'Provider Business Practice Location Address Postal Code',
                      'Healthcare Provider Taxonomy Code_1',
                      'Healthcare Provider Primary Taxonomy Switch_1',
                      'Healthcare Provider Taxonomy Code_2',
                      'Healthcare Provider Primary Taxonomy Switch_2',
                      'Healthcare Provider Taxonomy Code_3',
                      'Healthcare Provider Primary Taxonomy Switch_3',
                      'Healthcare Provider Taxonomy Code_4',
                      'Healthcare Provider Primary Taxonomy Switch_4',
                      'Healthcare Provider Taxonomy Code_5',
                      'Healthcare Provider Primary Taxonomy Switch_5',
                      'Healthcare Provider Taxonomy Code_6',
                      'Healthcare Provider Primary Taxonomy Switch_6',
                      'Healthcare Provider Taxonomy Code_7',
                      'Healthcare Provider Primary Taxonomy Switch_7',
                      'Healthcare Provider Taxonomy Code_8',
                      'Healthcare Provider Primary Taxonomy Switch_8',
                      'Healthcare Provider Taxonomy Code_9',
                      'Healthcare Provider Primary Taxonomy Switch_9',
                      'Healthcare Provider Taxonomy Code_10',
                      'Healthcare Provider Primary Taxonomy Switch_10',
                      'Healthcare Provider Taxonomy Code_11',
                      'Healthcare Provider Primary Taxonomy Switch_11',
                      'Healthcare Provider Taxonomy Code_12',
                      'Healthcare Provider Primary Taxonomy Switch_12',
                      'Healthcare Provider Taxonomy Code_13',
                      'Healthcare Provider Primary Taxonomy Switch_13',
                      'Healthcare Provider Taxonomy Code_14',
                      'Healthcare Provider Primary Taxonomy Switch_14',
                      'Healthcare Provider Taxonomy Code_15',
                      'Healthcare Provider Primary Taxonomy Switch_15']

# List of columns to pivot NPPES dataset on.
pivot_cols = ['Healthcare Provider Taxonomy Code_1',
            'Healthcare Provider Primary Taxonomy Switch_1',
            'Healthcare Provider Taxonomy Code_2',
            'Healthcare Provider Primary Taxonomy Switch_2',
            'Healthcare Provider Taxonomy Code_3',
            'Healthcare Provider Primary Taxonomy Switch_3',
            'Healthcare Provider Taxonomy Code_4',
            'Healthcare Provider Primary Taxonomy Switch_4',
            'Healthcare Provider Taxonomy Code_5',
            'Healthcare Provider Primary Taxonomy Switch_5',
            'Healthcare Provider Taxonomy Code_6',
            'Healthcare Provider Primary Taxonomy Switch_6',
            'Healthcare Provider Taxonomy Code_7',
            'Healthcare Provider Primary Taxonomy Switch_7',
            'Healthcare Provider Taxonomy Code_8',
            'Healthcare Provider Primary Taxonomy Switch_8',
            'Healthcare Provider Taxonomy Code_9',
            'Healthcare Provider Primary Taxonomy Switch_9',
            'Healthcare Provider Taxonomy Code_10',
            'Healthcare Provider Primary Taxonomy Switch_10',
            'Healthcare Provider Taxonomy Code_11',
            'Healthcare Provider Primary Taxonomy Switch_11',
            'Healthcare Provider Taxonomy Code_12',
            'Healthcare Provider Primary Taxonomy Switch_12',
            'Healthcare Provider Taxonomy Code_13',
            'Healthcare Provider Primary Taxonomy Switch_13',
            'Healthcare Provider Taxonomy Code_14',
            'Healthcare Provider Primary Taxonomy Switch_14',
            'Healthcare Provider Taxonomy Code_15',
            'Healthcare Provider Primary Taxonomy Switch_15']

# Dictionary for renaming columns in merged dataset.
rename_cols_dict = {'provider_organization_name_(legal_business_name)':'organization_name',
                    'provider_name_prefix_text':'name_prefix',
                    'provider_first_name':'first_name',
                    'provider_middle_name':'middle_name',
                    'provider_last_name_(legal_name)':'last_name',
                    'provider_name_suffix_text':'name_suffix',
                    'provider_credential_text':'credentials',
                    'provider_first_line_business_practice_location_address':'address_first_line',
                    'provider_second_line_business_practice_location_address':'address_second_line',
                    'provider_business_practice_location_address_city_name':'city',
                    'provider_business_practice_location_address_state_name':'state',
                    'provider_business_practice_location_address_postal_code':'zipcode',
                    'usps_zip_pref_city':'cbsa_city',
                    'usps_zip_pref_state':'cbsa_state',
                    'code':'taxonomy_code',
                    'grouping':'taxonomy_code_grouping',
                    'classification':'taxonomy_code_classification',
                    'specialization':'taxonomy_code_specialization',
                    'definition':'taxonomy_code_definition',
                    'display_name':'taxonomy_code_display_name',
                    'section':'taxonomy_code_section'}


# (preliminary) Final column ordering.
final_column_ordering = ['npi', 
                         'entity_type_code', 
                         'organization_name',
                         'name_prefix', 
                         'first_name', 
                         'middle_name',
                         'last_name',
                         'name_suffix',
                         'credentials', 
                         'address_first_line', 
                         'address_second_line', 
                         'city',
                         'state', 
                         'zipcode', 
                         'cbsa',
                         'cbsa_city',
                         'cbsa_state',
                         'taxonomy_code',
                         'taxonomy_code_grouping',
                         'taxonomy_code_classification',
                         'taxonomy_code_specialization',
                         'taxonomy_code_display_name',
                         'taxonomy_code_section']


# Read in taxonomy code classification dataset.
classifications = pd.read_csv('data/nucc_taxonomy_220.csv')

# Read in zip to cbsa dataset. Change zipcode column to string for ease of merge later.
cbsa = pd.read_csv('data/ZIP_CBSA_122021.csv', 
                   usecols = ['zip',
                              'cbsa',
                              'usps_zip_pref_city',
                              'usps_zip_pref_state',
                              'tot_ratio'])

cbsa['zip'] = cbsa['zip'].astype(str)

# Select rows where tot_ratio of cbsa to zip is the max for that zip.
cbsa = cbsa[cbsa.groupby(['zip'])['tot_ratio'].transform(max) == cbsa['tot_ratio']]

In [4]:
# Read in chunks from nppes csv file as chunks.
header = True
for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20220213.csv', 
                              usecols = list_of_nppes_cols, 
                              chunksize = 100000,
                             low_memory = False)):
    
    # Filter chunk for practice locations in TN only.
    chunk = chunk[chunk['Provider Business Practice Location Address State Name'] == 'TN']
    

    # Create two subsets of nppes dataset pivoted by NPI with taxonomy switches and codes and then concatenate together into
    # larger subset.
    chunk_taxonomy_code_subset = (
        pd.concat([chunk.melt(id_vars = 'NPI',
                              value_vars = pivot_cols[0:29:2],
                              var_name = 'code_number',
                              value_name = 'code')\
                   .sort_values(by = 'NPI'),
                   chunk.melt(id_vars = 'NPI',
                              value_vars = pivot_cols[1:29:2],
                              var_name = 'switch_number',
                              value_name = 'switch')\
                   .sort_values(by = 'NPI')],
                  axis = 1)
              )

    # Merge the subset dataset back into the larger one.
    chunk = (
        chunk\
        .drop(pivot_cols, axis = 1)\
        .merge(chunk_taxonomy_code_subset[['NPI','code']]\
        .loc[chunk_taxonomy_code_subset['switch'] == 'Y']\
        .iloc[:,[0,2]]\
        .sort_values('NPI')\
        .set_index('NPI')\
        .reset_index(), 
               how = 'inner', 
               on = 'NPI')
             )
    
    # Change zipcode to string to trim.
    chunk['Provider Business Practice Location Address Postal Code'] = (
        chunk['Provider Business Practice Location Address Postal Code']\
        .astype(str)\
        .apply(lambda x: x[0:5])
    )
    
    # Merge cbsa and classifications dataset to the chunk.
    chunk = (
        chunk.merge(classifications,
                    left_on = 'code',
                    right_on = 'Code',
                    how = 'inner')\
        .merge(cbsa,
               left_on = 'Provider Business Practice Location Address Postal Code',
               right_on = 'zip',
               how = 'inner')\
        .drop(columns = ['Code',
                         'Notes',
                         'zip'])
    )
    
    # Rename and reorder all columns.
    chunk.columns = [column.lower().replace(' ', '_') for column in chunk.columns]
    
    chunk = (
        chunk\
        .rename(columns = rename_cols_dict)
    )
    
    chunk = chunk[final_column_ordering]
    
    # Read chunk into sqlite database.
    chunk.to_sql('nppes', 
                 db, 
                 if_exists = 'append', 
                 index = False)
    
    # Write chunk to csv.
    chunk.to_csv('data/nppes.csv', 
                 header = header, 
                 mode = 'a', 
                 index = False)
    header = False

0it [00:00, ?it/s]

In [5]:
# Read back in full nppes dataset but only two rows--npi and entity type code.
only_npi_and_et = pd.read_csv('data/nppes.csv', 
                              usecols = ['npi','entity_type_code'], 
                              dtype = ('int','int'))

# Split these into two datasets for each npi column in hopteaming.
from_npi_et = (
    only_npi_and_et[['npi','entity_type_code']]\
    .rename(columns = {'entity_type_code':'from_npi_et'})\
    .set_index('npi')
)
to_npi_et = (
    only_npi_and_et[['npi','entity_type_code']]\
    .rename(columns = {'entity_type_code':'to_npi_et'})\
    .set_index('npi')
)

In [6]:
# Read in chunks from hop_teaming csv file. 
header = True
for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 100000)):
    
    # Merge chunk with npi entity codes.
    chunk = (
        chunk\
        .set_index('from_npi')\
        .join(from_npi_et,
              how = 'inner')\
        .reset_index()\
        .rename(columns = {'index':'from_npi'})\
        .set_index('to_npi')\
        .join(to_npi_et,
              how = 'inner')\
        .reset_index()\
        .rename(columns = {'index':'to_npi'})
       )
    
    # Filter based on entity codes.
    chunk = (
        chunk[(chunk['to_npi_et'] == 2) & (chunk['from_npi_et'] == 1)]\
        .drop(columns = ['to_npi_et','from_npi_et'])
    )
    
    # Filter for transaction count and average day wait and read to sqlite.
    chunk = chunk[(chunk['transaction_count'] >= 50) | (chunk['average_day_wait'] < 50)]
    
    chunk.to_sql('hopteaming', 
                 db, 
                 if_exists = 'append', 
                 index = False)  
    
    # Write chunk to csv.
    chunk.to_csv('data/hopteaming.csv', 
                 header = header, 
                 mode = 'a', 
                 index = False)
    header = False

0it [00:00, ?it/s]